In [175]:
from dotenv import load_dotenv
_ = load_dotenv()

In [176]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, ChatMessage, SystemMessage



In [177]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [178]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_completion_tokens=2048)

In [179]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level professional outline of an essay. \
Write such an professional outline for the user provided topic. Give an professional outline of the essay along with any relevant notes \
or instructions for the sections."""

In [180]:
WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays. \
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed:

------

{content}"""

In [181]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provided detailed recommendations, including requests for length, depth, sytle, funnyness, etc."""

In [182]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""

In [183]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outline below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""

In [184]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [185]:
from tavily import TavilyClient
import os

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [ ]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state['task'])
        
    ]
    response = llm.invoke(messages)
    return {"plan": response.content}

In [187]:
def research_plan_node(state: AgentState):
    queries = llm.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])

    content = state.get('content', [])
    for q in queries.queries:
        response = tavily.search(q, max_results=3)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}


In [188]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\n Here is my plan:\n\n{state['plan']}"
    )
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_message
    ]
    response = llm.invoke(messages)
    return {
        "draft": response.content,
        "revision_number": state.get("revision_number", 1) + 1
    }

In [189]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state['draft'])
    ]
    response = llm.invoke(messages)
    return {"critique": response.content}

In [190]:
def research_critique_node(state: AgentState):
    queries = llm.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(q, max_results=3)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [191]:
def should_continue(state: AgentState):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [192]:
builder = StateGraph(AgentState)

In [193]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [194]:
builder.set_entry_point("planner")

In [195]:
builder.add_conditional_edges("generate", should_continue, {END: END, "reflect": "reflect"})

In [196]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")


In [197]:
with SqliteSaver.from_conn_string(":memory:") as memory:
    graph = builder.compile(checkpointer=memory)
    thread = {"configurable": {"thread_id": "1"}}
    for s in graph.stream({
        "task": "What is the difference between langchain and langsmith",
        "max_revisions": 2,
        "revision_number": 1,
    }, thread):
        print(s)

{'planner': {'plan': 'I. Introduction\n    A. Brief overview of langchain and langsmith\n    B. Importance of understanding the differences between the two\n\nII. Langchain\n    A. Definition and explanation\n    B. Key characteristics\n    C. Use cases and applications\n\nIII. Langsmith\n    A. Definition and explanation\n    B. Key characteristics\n    C. Use cases and applications\n\nIV. Differences between Langchain and Langsmith\n    A. Technology\n    B. Functionality\n    C. Security\n    D. Scalability\n\nV. Conclusion\n    A. Recap of key points\n    B. Importance of choosing the right technology for specific needs\n    C. Future implications and advancements in the field of language technologies'}}
{'research_plan': {'content': ['LangChain vs LangSmith: Understanding the Differences, Pros, and Cons | by Ajay Verma | GoPenAI LangChain and LangSmith are two powerful tools developed by LangChain, a company focused on making it easier to build and deploy Large Language Model (LLM

ImportError: cannot import name 'ewriter' from 'helper' (/home/farzin/AI/Agentic-ai/env/lib/python3.12/site-packages/helper/__init__.py)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for helper: filename=helper-2.5.0-py2.py3-none-any.whl size=19220 sha256=f59ca188e48e730acdf743e0d7c8935b329e82a5782fb2fe0874aca504558518
  Stored in directory: /home/farzin/.cache/pip/wheels/7a/58/02/457999369b4271ecfffaa3d7a67d4501e3ca8ae97bca785ab8
Successfully built helper

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
